Выбираем корпус отзывов на товары одной из категорий Amazon и готовим аналитический отчет по этим отзывам — например, для производителя нового продукта этой категории. Для этого будем искать упоминания товаров в отзывах (будем считать их NE). Учтите, что упоминание может выглядеть не только как "Iphone 10", но и как "модель", "телефон" и т.п.

ячейка для импортов

In [4]:
import pandas as pd
import numpy as np
import os
import json
import gzip
from collections import Counter
import re
from string import punctuation
import math
import random

import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

## 0. Извлечение данных

сегодня мы будем смотреть на неожиданный датасет "подписки на журналы". Создатели датасета Амазон были так любезны, что поделились ячейкой кода для открытия этих архивов, вот ею и воспользуемся:

In [17]:
data = []
with gzip.open('Magazine_Subscriptions.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()).get('reviewText'))

print(len(data))

print(data[0])

data = [x for x in data if x!=None]

print(len(data))

89689
for computer enthusiast, MaxPC is a welcome sight in your mailbox. i can remember for years savorying every page of "boot" (as it was called in beginning) as i was (and still am) obcessed with PC's. Anyone, from advanced users - to beginners looking for knowledge - can profit from every issue of MaxPC. the icing on the cake is the subscription that comes with a CD-ROM as it is packed with demos, utilities, and other useful apps (very helpful for those not blessed with broadband connections). Until I discovered the community of hardware enthusiast web sites, MaxPC, formerly "boot", was my only really informative source for computing news and articles. To this day, i consider my subscription to it worth more than 10 subscriptions to most other computing mags. I can't wait until they merge with DVD media and maybe end up offering more info on Divx codecs, encoding your own movies, and best bang for the buck audio and video equipment. Try a few issues (with CD)and you may get hooked.

выглядит...правдоподобно.

## 1. Теоретические вопросы

Что мы можем сделать для извлечения?
* составить шаблоны для сочетаний слов, в которых можно найти нужные НЕР. Из плюсов - легко, быстро для машинного поиска и гарантированно правила будут иметь смысл, потому что мы сами ручками их задали, глазками посмотрев на энное количество отзывов. Из минусов? На все 90 тысяч отзывов не посмотришь, а правила выделения по количеству будут быстро расти.
* пойти другим путем и составить шаблоны частей речи, чаще всего являющиеся соседями наших НЕР. Похоже на предыдущий вариант, только правил придется делать куда меньше, но отсев кандидатов придется делать значительно более жесткий
* выделить энное количество (например, сотню-две) триграмм с ближайшими контекстами для настоящих и проверенных НЕР, векторизовать и триграммы, и отдельно контексты без НЕР, сравнивать кандидатные триграммы по близости с этим золотым стандартом

## 2. Извлечение сущностей

Мое сердце лежит скорее к шаблонам, так что их и попробуем построить. Но для начала - может, не будем зазря дергать Наташу, пока русским языком не занимаемся? Для английского чуть быстрее будет работать Спейси, а синтаксис правил примерно такой же.

Какие шаблоны можно выделить с первого взгляда на данные?
- Capital+Magazine
- read+Capital
- subscribe+to+Capital
- issue+of+х
- the+x+Magazine

напишем для них правила, но сначала сделаем себе вспомогательную функцию для хорошей токенизации и отделения знаков препинания:

In [16]:
def preprocess(text):
    tokens = re.split(' |\!|\?|\.|\,|\\n', text)
    tokens = [token.strip(punctuation) for token in tokens if token!='']
    return tokens

Можно строить!

Пайплайн у нас будет такой: создаем объект Спейси, который хранит в себе наши шаблоны и умеет их искать, после чего из каждого отзыва вытаскиваем все кандидатные НЕР, делаем их ключами словаря, а в качестве значений вписываем им список из всех отзывов, где они были найдены

In [765]:
patterns = [
    [{"LOWER": "the"}, {"OP": "+", "IS_ALPHA": True}, {"TEXT": "Magazine"}],
    [{"LEMMA": "issue"}, {"LOWER": "of"}, {"LOWER": "the", "OP": "?"}, {"TEXT": {"REGEX": "[QWERTYUIOPASDFGHJKLZXCVBNM].+"}, "OP": "+"}],
    [{"LEMMA": "read"}, {"LOWER": "the", "OP": "?"}, {"TEXT": {"REGEX": "[QWERTYUIOPASDFGHJKLZXCVBNM].+"},"OP": "+"}],
    [{"IS_LOWER": False, "IS_ALPHA": True}, {"LOWER": "magazine"}],
    [{"LEMMA": "subscribe"}, {"LOWER": "to"}, {"LOWER": "the", "OP": "?"}, {"IS_LOWER": False, "OP": "+", "IS_ALPHA": True}],
]
matcher.add("magazine", patterns, greedy='FIRST')

ner_dict = {}
for d in data:
    doc = nlp(d)
    ner = ''
    matches = matcher(doc)
    titleset = set()
    for match_id, start, end in matches:
        span = doc[start:end]
        if len((span.text).split(' '))<7 and len(span.text)>1 and span.text[:2].lower()!='th':
            pre_ner = re.split(r'issue.? of |subscri.{1,5} to |read.{0,3}[of]{0,3} ', span.text)
            if len(pre_ner)>1:
                workpiece = pre_ner[1].lower()
            else:
                workpiece = span.text.lower()
            ner = ' '.join(preprocess(workpiece))
            if ner not in ner_dict:
                ner_dict[ner] = [d.lower()]
            elif d.lower()!='':
                ner_dict[ner].append(d.lower())

In [766]:
len(ner_dict)

2170

выглядит ничего! давайте зальем в датафрейм и с ним уже и будем работать в дальнейшем

In [767]:
ner_frame = pd.DataFrame.from_dict(ner_dict, orient='index')

## 3. Н-граммы

Во-первых, мы сейчас сразу же получим в лицо двумя вещами: тем, что названия журналов могут состоять из скольких угодно слов, так что автовыделитель нграмм в нлтк нам не помощник, и словарем, в котором значениями являются списки списков, что усложнит нам жизнь. С первым, ну, ничего не поделаешь, будем руками нграммы вытаскивать, а вот второе вполне решаемо - напишем маленькую функцию, чтобы нормально расплющивать списки в однослойные:

In [60]:
def flatten(t):
    return [item for sublist in t for item in sublist]

стало спокойнее жить!

создаем новые списки, потому что жить спокойно нам не интересно. На этот раз механизм такой: для каждого выделенного НЕР ищем его конкретные вхождения в записанных на него отзывах, выдергиваем оттуда его левого и правого соседей, записываем в список левых и правых соседей соответственно...ну да, опять список списков. Но это имеет смысл - мы же для каждого НЕР свой подсписок формируем, чтобы ничего не перепуталось.

In [804]:
ngrams_left = []
ngrams_right = []
for row in ner_frame.index:
    row_ngrams_left = []
    row_ngrams_right = []
    for cell in ner_frame.loc[row]:
        if cell!=None:
            row_ngrams_left.append([inst.replace(' '+row, '') for inst in re.findall(r'[A-Za-z]+\b'+' '+re.escape(row), cell)])
            row_ngrams_right.append([inst_1.replace(row+' ', '') for inst_1 in re.findall(re.escape(row)+r' [A-Za-z]+\b', cell)])
        else:
            row_ngrams_left.append([''])
            row_ngrams_right.append([''])
    ngrams_left.append([w for w in flatten(row_ngrams_left) if w!=''])
    ngrams_right.append([w for w in flatten(row_ngrams_right) if w!=''])

правые соседи выглядят неплохо. вкинем их в датафрейм и посмотрим, что там как.

In [ ]:
ngrams_frame = pd.DataFrame([ner_frame.index, ngrams_left, ngrams_right]).T.set_index(0, drop=True)
ngrams_frame.columns = ['left_context', 'right_context']

## 4. ранжирование коллокаций

результаты получились заметно странноватые, так что будем их ранжировать.

Для начала, давайте сделаем функцию, которая будет нам считать количество раз, когда наша н-грамма встретилась во всех отзывах вообще. Чтобы не дробить хоть эту функцию на две, будем просто сообщать ей, левый или правый сосед НЕР нас интересует.

In [843]:
def ngram_freq(ngrams, row, text, flag):
    dict_row = {}
    for gram in ngrams:
        if flag == 'left':
            insta = gram+' '+row
        elif flag == 'right':
            insta = row+' '+gram
        loc_ngrams = re.findall(r'\b'+re.escape(insta)+r'\b', text)
        dict_row[insta] = len(loc_ngrams)
    
    return dict_row

Пора, пора! Пора воплотить в жизнь наши планы и посчитать количество для каждой н-граммы. Заведем для этого что? Словарь или список списков? - у меня не очень большая фантазия. 

Словарь, на этот раз словарь.

In [844]:
ngrams_left_dict = {}
ngrams_right_dict = {}
all_revs = ' '.join(data).lower()
for mag in ngrams_frame.index:
    ngrams_left_dict.update(ngram_freq(ngrams_frame.loc[mag]['left_context'], mag, all_revs ,flag='left'))
    ngrams_right_dict.update(ngram_freq(ngrams_frame.loc[mag]['right_context'], mag, all_revs ,flag='right'))

И результат закатаем во что? Правильно, в датасет. Я такой многогранный специалист.

Сначала сделаем левых соседей, потом правых.

In [1084]:
ngrams_freq_left = pd.DataFrame.from_dict(ngrams_left_dict, orient='index')
ngrams_freq_left.reset_index(inplace=True)
ngrams_freq_left.columns = ['bigram', 'num']

Воспользуемся тем, что левым соседом у нас является просто одно отделенное пробелом слово по условию, и разобьем наши биграммы просто единожды сплитнув слева по пробелу.

In [1086]:
ngrams_freq_left[['word_1', 'word_2']] = ngrams_freq_left.bigram.str.split(' ', n=1, expand=True)

почистим от знаков препинания и больших букв наши данные и сохраним в отдельную переменную, чтобы каждый раз 90 тысяч отзывов не ворочать; для полученной длинной-длинной строки подсчитаем количество токенов простым сплитом по пробелу, потому что все знаки препинания мы уже удалили

In [18]:
data_prep = ' '.join(preprocess(' '.join(data).lower()))

In [19]:
all_token_len = len(data_prep.split(' '))

Теперь для метрик нам понадобится страшное дело - посчитать все возможные биграммы для этого текста. Зачем снова брать чистый непредобработанный текст со всеми знаками препинания, на кой мы это делаем? В основном затем, чтобы как возможные биграммы у нас не записались слова, в оригинале относящиеся к разным предложениям и потому разделенные точкой или каким-нибудь восклицательным знаком. Поэтому здесь для токенизации мы используем отдельный модуль нлтк, который при сплите выделяет как отдельные токены знаки препинания, а потом просто не учитываем биграммы с ними при подсчете общего числа возможных биграмм.

In [20]:
def bigram_sum(text):
    pred = ' '.join(re.split(r'\\n', text))
    tokens = wordpunct_tokenize(pred)
    i = 0
    for each in range(len(tokens)-1):
        if tokens[each+1] not in punctuation and tokens[each] not in punctuation:
            i = i+1
    return i

In [21]:
all_bigrams = bigram_sum(' '.join(data))

продолжаем собирать данные, которые нам пригодятся в определении метрик. На очереди частотность. Для соседей это определить просто - найди все случаи по корпусу, подели на все токены, - а вот с НЕР мы в лобовую сталкиваемся с тем, что большинство наших НЕР сами являются композитами из двух и даже трех токенов. На что их делить? Давайте на общее число биграмм, всё чище будет

In [1088]:
ngrams_freq_left['freq_word_1'] = [len(re.findall(r'\b'+ re.escape(word)+r'\b', data_prep))/all_token_len for word in ngrams_freq_left['word_1']]

In [1092]:
ngrams_freq_left['freq_word_2'] = [len(re.findall(r'\b'+re.escape(word)+r'\b', data_prep))/all_bigrams for word in ngrams_freq_left['word_2']]

Выкинем случаи, где у нас встретились кандидаты, которых фактически и нет

In [ ]:
ngrams_freq_left = ngrams_freq_left[ngrams_freq_left.num > 0]

In [ ]:
ngrams_freq_left = pd.read_csv('ngrams_left_freq', encoding='utf-8', index_col=0)
ngrams_freq_left

И вот мы наконец готовы считать меры ассоциации! Возьмем Т, Дайса и ПМИ и сделаем для них общую функцию, возвращающую словарь с их результатами для каждой нграммы:

In [118]:
def colloc_metrics(ngram_count, len_set, all_token_len, all_bigrams, freq_word_1, freq_word_2):
    norm_freq = (ngram_count/all_bigrams)/len_set
    mu = freq_word_1*freq_word_2
    fin_dict = {}
    fin_dict['t_score'] = (norm_freq-mu)/(norm_freq**0.5)
    fin_dict['dice'] = (2*norm_freq)/(freq_word_1+freq_word_2)
    fin_dict['pmi'] = math.log(norm_freq/mu)
    return fin_dict

и, собственно, применим ее для каждой нграммы

In [119]:
t_scores = []
dices = []
pmis = []
for i in ngrams_freq_left.index:
    currow = ngrams_freq_left.loc[i]
    metric_dict = colloc_metrics(currow['num'], len(ngrams_freq_left), all_token_len, all_bigrams, currow['freq_word_1'], currow['freq_word_2'])
    t_scores.append(metric_dict['t_score'])
    dices.append(metric_dict['dice'])
    pmis.append(metric_dict['pmi'])

In [120]:
ngrams_freq_left = ngrams_freq_left.assign(T_score = t_scores, Dice = dices, PMI = pmis)

In [121]:
ngrams_freq_left.head()

,bigram,num,word_1,word_2,freq_word_1,freq_word_2,T_score,Dice,PMI
0,of maxpc,1,of,maxpc,0.019446,0.000004,-0.010019,5.520308e-09,-7.221377
1,at antiques magazine,1,at,antiques magazine,0.003218,0.000002,-0.000758,3.334853e-08,-4.649259
2,love antiques magazine,1,love,antiques magazine,0.004335,0.000002,-0.001024,2.475863e-08,-4.947238
3,fine antiques magazine,1,fine,antiques magazine,0.000262,0.000002,-0.000055,4.066117e-07,-2.142331
4,read maximum pc,1,read,maximum pc,0.004208,0.000030,-0.017347,2.533594e-08,-7.770065


красота! но вот эти предлоги в качестве соседей несколько смущают. Выбросим отсюда стоп-слова.

In [123]:
stoplist = stopwords.words('english')
ngrams_freq_left_stopped = ngrams_freq_left[~ngrams_freq_left['word_1'].isin(stoplist)]

и вот теперь посмотрим для каждой метрики на ее любимых кандидатов в нграммы

In [124]:
t_score_left_stopped = ngrams_freq_left_stopped.sort_values(by=['T_score'])[:50]
dice_left_stopped = ngrams_freq_left_stopped.sort_values(by=['Dice'])[:50]
pmi_left_stopped = ngrams_freq_left_stopped.sort_values(by=['PMI'])[:50]

In [125]:
t_score_left_stopped.drop_duplicates('word_1')

,bigram,num,word_1,word_2,freq_word_1,freq_word_2,T_score,Dice,PMI
4901,read issue,5,read,issue,0.004208,0.004285,-1.100439,6.321256e-08,-11.114964
1941,reading kindle,1,reading,kindle,0.002256,0.002467,-0.759687,2.273135e-08,-11.549119
1946,love kindle,6,love,kindle,0.004335,0.002467,-0.595933,9.470504e-08,-10.410481
1269,get time,7,get,time,0.002969,0.002664,-0.408112,1.334033e-07,-9.954837
1274,like time,21,like,time,0.004299,0.002664,-0.341089,3.238061e-07,-9.226183
175,magazine good magazine,27,magazine,good magazine,0.017732,0.000491,-0.228831,1.590726e-07,-8.701436
258,would a magazine,2,would,a magazine,0.002658,0.000839,-0.215202,6.140403e-08,-9.941254
2277,find money,1,find,money,0.001563,0.000964,-0.205740,4.247955e-08,-10.242850
729,enjoy world,1,enjoy,world,0.001456,0.001011,-0.200890,4.352421e-08,-10.218998
1273,think time,4,think,time,0.001031,0.002664,-0.187388,1.162266e-07,-9.456315


ну...ничего, в принципе. Он действительно подметил много слов, по которым и мы искали кандидатные НЕР, когда создавали правила. 

In [126]:
dice_left_stopped.drop_duplicates('word_1')

,bigram,num,word_1,word_2,freq_word_1,freq_word_2,T_score,Dice,PMI
1914,magazine country living,1,magazine,country living,0.017732,0.000060,-0.144868,6.034470e-09,-9.892073
401,great history magazine,1,great,history magazine,0.005969,0.000017,-0.013961,1.793451e-08,-7.552998
4474,read times,1,read,times,0.004208,0.000614,-0.352593,2.226825e-08,-10.781538
2467,articles writer,1,articles,writer,0.004627,0.000133,-0.083673,2.255841e-08,-9.343205
1941,reading kindle,1,reading,kindle,0.002256,0.002467,-0.759687,2.273135e-08,-11.549119
2525,love food magazine,1,love,food magazine,0.004335,0.000029,-0.017185,2.460366e-08,-7.760649


впечатляюще он любит одни и те же варианты как лучших соседей выдавать! Впрочем, может, это и к лучшему - он очень неплохо справился с выделением и практически нигде я с ним не могу поспорить.

In [127]:
pmi_left_stopped.drop_duplicates('word_1')

,bigram,num,word_1,word_2,freq_word_1,freq_word_2,T_score,Dice,PMI
1941,reading kindle,1,reading,kindle,0.002256,0.002467,-0.759687,2.273135e-08,-11.549119
4901,read issue,5,read,issue,0.004208,0.004285,-1.100439,6.321256e-08,-11.114964
1946,love kindle,6,love,kindle,0.004335,0.002467,-0.595933,9.470504e-08,-10.410481
2277,find money,1,find,money,0.001563,0.000964,-0.205740,4.247955e-08,-10.242850
729,enjoy world,1,enjoy,world,0.001456,0.001011,-0.200890,4.352421e-08,-10.218998
1269,get time,7,get,time,0.002969,0.002664,-0.408112,1.334033e-07,-9.954837
258,would a magazine,2,would,a magazine,0.002658,0.000839,-0.215202,6.140403e-08,-9.941254
1914,magazine country living,1,magazine,country living,0.017732,0.000060,-0.144868,6.034470e-09,-9.892073
1270,gives time,1,gives,time,0.000340,0.002664,-0.123585,3.573805e-08,-9.733189
246,way a magazine,1,way,a magazine,0.001077,0.000839,-0.123257,5.605225e-08,-9.730538


Неплохо! Очень ему понравились частые глаголы. Но в принципе, тоже неплох. Сложно сказать, кто лучше для выделения наших НЕР - точность выше у Дайса, но полнота у остальных двоих. Пожалуй, наилучший баланс точности и полноты я отмечу у ПМИ.

А теперь все то же самое, но для правого контекста! Пусть НЕР остаются с названием "слово_2", чтобы не путаться, и на этот раз сплит биграммы используем с правой стороны

In [90]:
ngrams_freq_right = pd.DataFrame.from_dict(ngrams_right_dict, orient='index')
ngrams_freq_right.reset_index(inplace=True)
ngrams_freq_right.columns = ['bigram', 'num']
ngrams_freq_right[['word_2', 'word_1']] = ngrams_freq_right.bigram.str.rsplit(' ', n=1, expand=True)
ngrams_freq_right['freq_word_1'] = [len(re.findall(r'\b'+ re.escape(word)+r'\b', data_prep))/all_token_len for word in ngrams_freq_right['word_1']]
ngrams_freq_right['freq_word_2'] = [len(re.findall(r'\b'+ re.escape(word)+r'\b', data_prep))/all_bigrams for word in ngrams_freq_right['word_2']]
ngrams_freq_right = ngrams_freq_right[ngrams_freq_right.num > 0]

In [114]:
ngrams_freq_right.head()

,bigram,num,word_2,word_1,freq_word_1,freq_word_2
0,maxpc is,4,maxpc,is,0.015617,0.000004
1,antiques magazine is,3,antiques magazine,is,0.015617,0.000002
2,antiques magazine because,1,antiques magazine,because,0.001417,0.000002
3,maximum pc app,1,maximum pc,app,0.000325,0.000030
4,maximum pc in,1,maximum pc,in,0.011943,0.000030


In [129]:
t_scores_r = []
dices_r = []
pmis_r = []
for i in ngrams_freq_right.index:
    currow = ngrams_freq_right.loc[i]
    metric_dict = colloc_metrics(currow['num'], len(ngrams_freq_right), all_token_len, all_bigrams, currow['freq_word_1'], currow['freq_word_2'])
    t_scores_r.append(metric_dict['t_score'])
    dices_r.append(metric_dict['dice'])
    pmis_r.append(metric_dict['pmi'])
ngrams_freq_right = ngrams_freq_right.assign(T_score = t_scores_r, Dice = dices_r, PMI = pmis_r)
ngrams_freq_right_stopped = ngrams_freq_right_t[~ngrams_freq_right_t['word_1'].isin(stoplist)]
t_score_right_stopped = ngrams_freq_right_stopped.sort_values(by=['T_score'])[:50]
dice_right_stopped = ngrams_freq_right_stopped.sort_values(by=['Dice'])[:50]
pmi_right_stopped = ngrams_freq_right_stopped.sort_values(by=['PMI'])[:50]

очень оперативно, в пределах пары ячеек, провернули все то же самое и смотрим уже на скоры для правых соседей.

In [130]:
t_score_right_stopped.drop_duplicates('word_1')

,bigram,num,word_2,word_1,freq_word_1,freq_word_2,T_score,Dice,PMI
4741,who magazine,5,who,magazine,0.017732,0.002084,-2.275803,2.661336e-08,-11.850460
1557,time kindle,1,time,kindle,0.002188,0.002664,-0.802708,2.173752e-08,-11.613083
83,great magazine read,1,great magazine,read,0.004208,0.001367,-0.791792,1.892249e-08,-11.599391
1555,time one,6,time,one,0.003698,0.002664,-0.553854,9.947131e-08,-10.346137
1743,in amazon,31,in,amazon,0.001654,0.013469,-0.551001,2.162120e-07,-9.519900
104,great magazine would,1,great magazine,would,0.002658,0.001367,-0.500219,2.620689e-08,-11.140145
1558,time subscription,19,time,subscription,0.004224,0.002664,-0.355505,2.909321e-07,-9.326492
102,great magazine like,6,great magazine,like,0.004299,0.001367,-0.330241,1.117070e-07,-9.829080
1535,time cover,2,time,cover,0.001241,0.002664,-0.322063,5.400943e-08,-10.353288
136,great magazine really,2,great magazine,really,0.002017,0.001367,-0.268429,6.233954e-08,-10.171136


м, все в кучу собирает. Полнота неплохая, но предсказательная сила мне его не нравится - ухитрился выбрать самые сомнительные варианты НЕР вообще

In [131]:
dice_right_stopped.drop_duplicates('word_1')

,bigram,num,word_2,word_1,freq_word_1,freq_word_2,T_score,Dice,PMI
4443,the world magazine,1,the world,magazine,0.017732,0.000427,-1.04303,5.808329e-09,-11.874976


ха, в принципе, одобряю его решение! Он на все 50 своих топ-вариантов взял просто "журнал". Чисто технически, он просто зацепился за мой прокол - стоило автоматически добавлять слово "журнал" к каждому НЕР, где его еще нет, и тогда оно не попало бы в правых соседей НЕР, - так что винить его не могу, он просто эксплуатирует проблемы инпута, и делает это весьма неплохо. Из любопытства можем на это воочию посмотреть:

In [132]:
dice_right_stopped[:10]

,bigram,num,word_2,word_1,freq_word_1,freq_word_2,T_score,Dice,PMI
4443,the world magazine,1,the world,magazine,0.017732,0.000427,-1.043030,5.808329e-09,-11.874976
3606,cooks magazine,1,cooks,magazine,0.017732,0.000075,-0.183056,5.923206e-09,-10.134918
1321,analog magazine,1,analog,magazine,0.017732,0.000055,-0.134807,5.929786e-09,-9.828982
4320,science news magazine,1,science news,magazine,0.017732,0.000030,-0.073076,5.938226e-09,-9.216687
4075,wwii magazine,1,wwii,magazine,0.017732,0.000020,-0.049661,5.941434e-09,-8.830453
5438,japanese magazine,1,japanese,magazine,0.017732,0.000017,-0.042566,5.942406e-09,-8.676303
5674,skiing magazine,1,skiing,magazine,0.017732,0.000015,-0.035470,5.943380e-09,-8.493981
2577,running times magazine,1,running times,magazine,0.017732,0.000011,-0.027665,5.944450e-09,-8.245520
2363,read the economist magazine,1,read the economist,magazine,0.017732,0.000009,-0.022698,5.945132e-09,-8.047694
854,flex magazine,1,flex,magazine,0.017732,0.000008,-0.019860,5.945521e-09,-7.914163


Это и правда не баг, а фича. Забавно. Но действительно технически верно!

In [133]:
pmi_right_stopped.drop_duplicates('word_1')

,bigram,num,word_2,word_1,freq_word_1,freq_word_2,T_score,Dice,PMI
4443,the world magazine,1,the world,magazine,0.017732,0.000427,-1.043030,5.808329e-09,-11.874976
1557,time kindle,1,time,kindle,0.002188,0.002664,-0.802708,2.173752e-08,-11.613083
83,great magazine read,1,great magazine,read,0.004208,0.001367,-0.791792,1.892249e-08,-11.599391
104,great magazine would,1,great magazine,would,0.002658,0.001367,-0.500219,2.620689e-08,-11.140145
87,great magazine one,2,great magazine,one,0.003698,0.001367,-0.492055,4.165359e-08,-10.777121
130,great magazine still,1,great magazine,still,0.001328,0.001367,-0.249815,3.915017e-08,-10.445834
4442,the world subscription,1,the world,subscription,0.004224,0.000427,-0.248453,2.267749e-08,-10.440365
141,great magazine cover,1,great magazine,cover,0.001241,0.001367,-0.233619,4.044205e-08,-10.378808
4202,the time articles,1,the time,articles,0.004627,0.000334,-0.212540,2.126293e-08,-10.284249
1551,time seems,1,time,seems,0.000561,0.002664,-0.205800,3.270237e-08,-10.252024


журнал Time начинает действовать мне на нервы.

А если серьезно, что ж. Мне нравится логика Дайса, и я предполагаю, что он просто поставлен в условия, где входные данные сами предоставили ему возможность легко вычленить искомое, но это уже в дальнейшем можно было бы покрутить. По полноте с учетом частотности самой нграммы выше Т - возможно потому, что ему вообще частотные вещи нравятся. Проверим, запретив ему брать наименее частотные биграммы

In [137]:
ngrams_freq_right_stopped[ngrams_freq_right_stopped.num > 5].sort_values(by=['PMI'])[:50].drop_duplicates('word_1')

,bigram,num,word_2,word_1,freq_word_1,freq_word_2,T_score,Dice,PMI
3007,more magazine,27,more,magazine,0.017732,0.003668,-1.723532,1.330768e-07,-10.729318
1555,time one,6,time,one,0.003698,0.002664,-0.553854,9.947131e-08,-10.346137
102,great magazine like,6,great magazine,like,0.004299,0.001367,-0.330241,1.117070e-07,-9.829080
1743,in amazon,31,in,amazon,0.001654,0.013469,-0.551001,2.162120e-07,-9.519900
1558,time subscription,19,time,subscription,0.004224,0.002664,-0.355505,2.909321e-07,-9.326492
86,great magazine even,7,great magazine,even,0.001502,0.001367,-0.106794,2.574089e-07,-8.623213
1281,us magazines,8,us,magazines,0.002384,0.000825,-0.095706,2.629589e-07,-8.446854
111,great magazine always,13,great magazine,always,0.002032,0.001367,-0.106045,4.034253e-07,-8.306715
348,good magazine good,11,good magazine,good,0.004010,0.000491,-0.081789,2.577451e-07,-8.130569
133,great magazine love,35,great magazine,love,0.004335,0.001367,-0.137845,6.475011e-07,-8.073848


в принципе, в среднем для правых и левых соседей ПМИ работает стабильнее, хоть и средне, так что я бы выбрала его.

## 5. Группировка по товарам

Нам, к счастью, сгруппировать все будет предельно просто - мы работаем в датафреймах, а они позволяют группировать по значению колонки. Сделаем это, увидим, что пандасы при группировке выдают индексы нужных строк, вздохнем, создадим итоговые словарики (последние, честно) и сделаем выделенные группы ключами словаря, а значения заменим с индексов на соседей по биграмме.

In [101]:
group_left_dict = ngrams_freq_left_stopped.groupby(by='word_2', sort=False).groups
group_right_dict = ngrams_freq_right_stopped.groupby(by='word_2', sort=False).groups

In [102]:
ner_left = {}
ner_right = {}
for i in set(ngrams_freq_left_stopped['word_2']):
    ner_left[i] = [ngrams_freq_left_stopped.loc[x]['word_1'] for x in group_left_dict.get(i)]
for j in set(ngrams_freq_right_stopped['word_2']):
    ner_right[j] = [ngrams_freq_right_stopped.loc[x]['word_1'] for x in group_right_dict.get(j)]

И все! Вот в этих итоговых словарях и лежат наши соседи, разбитые по своим НЕР. Давайте выведем случайные 5 правых и левых групп.

In [104]:
random.sample(ner_left.items(), 5)

C:\Users\BCD4~1\AppData\Local\Temp/ipykernel_9128/2622876490.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(ner_left.items(), 5)


[('the nation', ['read', 'await']),
 ('woodsmith magazine', ['last']),
 ('forbes magazine', ['love', 'purchased', 'liked', 'buy']),
 ('the food babe', ['read']),
 ('the world', ['read'])]

In [111]:
random.sample(ner_right.items(), 5)

C:\Users\BCD4~1\AppData\Local\Temp/ipykernel_9128/3558431674.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(ner_right.items(), 5)


[('click magazine', ['made']),
 ('vqr', ['material', 'features', 'decided']),
 ('romantic homes', ['company']),
 ('state magazine', ['covers']),
 ('archaeology magazine', ['shines', 'sticks'])]

какие выводы мы можем сделать из собранных нграмм?
* кажется, названия журналов заметно тяготеют к тому, чтобы иметь при себе глаголы: это можно будет учитывать в дальнейшей работе с выделением названий по шаблонам.
* названия журналов ничем особо не ограничены, так что даже я, человек, с трудом отличаю настоящие от ложно выделенных: действительно ничем не могу гарантировать, что нет журнала The или A, а википедия знает журналы Good, Print, O, и разные другие интересные вещи.
* из того, что можно было бы улучшить в будущих версиях: автоматически прикреплять к выделенному NER 'THE'+NER+'MAGAZINE', если этого там уже нет - унифицирует выдачу и позволит не смотреть на артикли и слово "журнал" в качестве биграмм, учитывая то, что мы по этим словам НЕР и выделяли в шаблонах; после опознания кандидата по словам типа "читать" или "журнал" вырезать эти слова из самого кандидата.